# Laboratorio 7: Poke-ensambles

**Duración:** 2 horas  
**Formato:** Competencia por equipos con rotaciones y
kahoots Pokemon  
**Repositorio base:** [_link al repo oficial_](https://github.com/JAlbertoAlonso/poke-ensambles/tree/main)

---

## Portada del equipo

**Integrantes:**
- Nombre 1 (Usuario GitHub)
- Nombre 2 (Usuario GitHub)
- Nombre 3 (Usuario GitHub)

**Repositorio del equipo:**  
<https://github.com/usuario/equipoX>

**Fecha de entrega:**  
__/__/____

## Elemento 1 - Construcción del dataset (Infraestructura)

## Elemento 2 - Entrenamiento de modelos base (Modelado)

## Elemento 3 - Ensambles de votación

## Elemento 4 - Evaluación y competencia

## Cuestionario teórico